# TRY Conference 2019 
## Usage Data Analysis Using Python 
### Presenters: Lei Jin, Josephine Choi 

### https://trylibraryconference.wordpress.com/presentation-descriptions-3/#session3pres6


### Synposis 
COUNTER data provided by publishers quantifies use by title for journals, databases, and ebooks. However, we are unable to see how widely spread or concentrated this use is amongst our University population, and lack details to inform our qualitative information gathering for interdisciplinary content. With the adoption of open source and Python technologies, this ongoing project aims to provide comprehensive in-depth e-resources usage reports by combining EZProxy usage data and student registration data. After final data integration and report generation, we are able to see the use of electronic resources by departments, by student profiles (year, program), thus provide collection development support to subject liaisons. This additional context is useful for decision making, marketing, and communication. It helps to answer questions concerning collection development, such as, what are the top 10 most popular resources used by one certain department? Which student group would be most affected with the cancellation of one certain resource? Or how does a certain department use the resources they requested? etc. The open source and Python scripts are easily shared for those who are interested and desire to adopt this methodology in their own settings.

Lei Jin Electronic Resources Librarian Collection, Ryerson University Library and Archives
Josephine Choi Library Technician – ER, Acquisition, and Serials Collection, Ryerson University Library and Archives

### Acknowledgement 

The code in this notebook is based on code authored by Petrina Collingwood to get "monthly/year domain hits/usage from Ezproxy logs". Collingwood's code counts usage stats per second, and thus "eliminates counting of excess lines in the log that were generated by one click and therefore makes the data more realistic" 

More about Collingwood's project can be found in her Github(https://github.com/prcollingwood/ezproxy)

### Using this notebook 
- We used Anaconda (Python 3.5) for this project 
- Modification based on ezproxy log file format will be required 
- Edit of ezproxy prefix will be required


In [1]:
# MODIFED FROM CODE BY Petrina Collingwood  


def domain_count(filename):
    
    import csv
    import re
    # create csv file from log file
    with open(filename,'r') as fh:
        with open('csv/' + filename + '.csv','w') as outfile:
            for line in fh:
                print(re.sub(r'\n|"','',line), file=outfile)
    import pandas as pd
    from urllib.parse import unquote
    # create dataframe from csv file skipping malformed lines
    df = pd.read_csv('csv/' + filename + '.csv',sep=' ', error_bad_lines=False, header=None, encoding='utf-8')
    # remove unnecessary columns
    df.drop(df.columns[[2,5,6,8,9]], axis=1, inplace=True)
    # name columns
    df.columns = ['ip', 'session_id', 'user_id', 'date_time', 'url', 'size']
    # formate date/time column
    df['date_time'] = df['date_time'].map(lambda x: x.lstrip('['))
    df['date_time'] = pd.to_datetime(df['date_time'], format='%d/%b/%Y:%H:%M:%S')
    # remove lines where user is not logged in
    df = df[df.user_id != "-"]
    # decode urls
    def decode_url(url):
        decoded_url = unquote(url)
        return decoded_url
    df['url'] = df.url.apply(decode_url)
    # remove excess columns for domain
    df.drop(['ip','session_id','size'], axis=1, inplace=True)
    # remove ezp string from start of url
    df['url'] = df['url'].str.replace(r'^http://ezproxy\.lib\.ryerson\.ca/login/\?url=', '')# remove http etc
    df['url'] = df['url'].str.replace(r'^http://www\.|^https://www\.|^http://|^https://', '')
    # remove ezproxy string from start of url
    def parse_url(url):
        if (url.startswith("ezproxy.lib.ryerson.ca/login?url=")) and ("http" in url):
            location = url.find("http")
            return url[location:]
        elif (url.startswith("ezproxy.lib.ryerson.ca/login?url=")):
            return "-"
        else:
            return url
    df['url'] = df.url.apply(parse_url)
    # remove http etc
    df['url'] = df['url'].str.replace(r'^http://www\.|^https://www\.|^http://|^https://', '')
    # remove rows where ezproxy string is the only url
    df = df[df.url != "-"]
    # remove spaces introduced by unquoting
    df['url'] = df['url'].str.replace(r'\n', '')
    # remove everything after : or / or ?
    df['url'] = df['url'].str.replace(r'[:/?].*$', '')
    # remove .ezp.lib.unimelb.edu.au from urls
    df['url'] = df['url'].str.replace(r'ezproxy\.lib\.ryerson\.ca', '')
    df['url'] = df['url'].str.replace(r'ezproxy\.lib\.', '-')
    df = df[df.url != "-"]
    # create new column of domains
    def get_domain(url):
        regexp = re.compile(r'\.com|\.org|\.net|\.edu|-org|-com|\.gov')
        if regexp.search(url) is not None:
            for match in regexp.finditer(url):
                location = match.start()
            new_url = url[:location]
            if ('.' in new_url):
                location = new_url.rfind('.')
            elif ('-' in new_url):
                location = new_url.rfind('-')
            else:
                return url
            location += 1
            
            if ("-org" in url[location:]):
                modified_url = url[location:].replace(r'-org', '.org')
            elif ("-com" in url[location:]):
                modified_url = url[location:].replace(r'-com', '.com')
            else:
                return url[location:]
            return modified_url
        else:
            return url
    df['domain'] = df.url.apply(get_domain)
    
    
    # remove duplicate rows which have same user_id, date-time and domain. 
    df.drop_duplicates(subset=['date_time','domain'], inplace=True)
   
    df.to_csv('student_count.csv',index=False, encoding='utf-8')
    


A folder needs to be created in csv in order for this to work 

This part of the code is to combined the simplified ezproxy file with student data

In [12]:
#THIS CODE WORKS !!!
import pandas as pd
import glob
import os

studentdf = pd.read_csv("data/student1819.csv")
newdf=pd.DataFrame()

# this needs to be changed for each month 

for filename in glob.iglob(os.path.join("offcampus/201808/","*.log")):
    filename = filename.replace("\\","/")
    # call log analysis function 
    domain_count(filename)
    # create dataframe from csv output file from daily domain count
    df=pd.read_csv("student_count.csv",sep=",")
    df_combined=pd.merge(df,studentdf,how="left",left_on="user_id",right_on="USERNAME")
    df_combined.columns=['user_id','date_time','url','domain','USERNAME',"acad_prog","prog_yr"]
    df_combined=df_combined.drop('USERNAME',axis=1)
    df_combined['acad_prog']=df_combined['acad_prog'].fillna("staff")
    df_combined['prog_yr']=df_combined['prog_yr'].fillna("Other")
    df_combined['on_or_off']="offcampus"
    newdf = newdf.append(df_combined)  
    newdf.to_csv('combined201808_offcampus.csv',index=False, encoding='utf-8')



b'Skipping line 1998: expected 11 fields, saw 19\nSkipping line 1999: expected 11 fields, saw 18\nSkipping line 2000: expected 11 fields, saw 18\nSkipping line 2001: expected 11 fields, saw 18\nSkipping line 2002: expected 11 fields, saw 17\nSkipping line 2003: expected 11 fields, saw 17\n'
